# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('../output_data/city.csv')
city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kaitangata,9,NZ,1619245895,74,-46.2817,169.8464,50.00,1.99
1,1,Shumskiy,100,RU,1619245896,87,54.8318,99.1328,35.38,4.68
2,2,Airai,86,TL,1619245896,74,-8.9266,125.4092,67.59,2.91
3,3,Cape Town,0,ZA,1619245896,82,-33.9258,18.4232,63.00,9.22
4,4,Kapaa,90,US,1619245897,73,22.0752,-159.3190,75.20,16.11
...,...,...,...,...,...,...,...,...,...,...
561,561,Gambēla,41,ET,1619246104,42,8.2500,34.5833,89.73,5.32
562,562,Lanzhou,90,CN,1619246104,66,36.0564,103.7922,48.20,4.47
563,563,Mocajuba,99,BR,1619246105,99,-2.5842,-49.5072,72.84,2.19
564,564,Pueblo Nuevo,100,MX,1619246105,85,17.1000,-92.8833,60.94,4.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [11]:
locations = city_df[['Lat','Lng']]
humidity = city_df["Humidity"]
fig = gmaps.figure(center = [0,0] ,zoom_level = 1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
ideal_city = city_df.loc[(city_df['Max Temp']>70) & (city_df['Max Temp']<80) & (city_df['Wind Speed']<10)
                        & (city_df['Cloudiness']==0)]
ideal_city

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
123,123,Lüderitz,0,NaN,1619245938,56,-26.6481,15.1594,71.76,5.66
125,125,Vila Velha,0,BR,1619245907,88,-20.3297,-40.2925,71.60,4.61
177,177,Chiredzi,0,ZW,1619245958,52,-21.0500,31.6667,72.66,1.83
290,290,Doaba,0,PK,1619245999,28,33.4244,70.7364,75.90,6.33
422,422,Atar,0,MR,1619246050,20,20.5169,-13.0499,73.54,9.57
449,449,Gwanda,0,ZW,1619246060,36,-20.9333,29.0000,71.06,5.41
514,514,Kalat,0,PK,1619246085,12,29.0225,66.5916,73.47,8.59
542,542,Saint-Louis,0,SN,1619246096,64,16.3333,-15.0000,71.60,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = ideal_city
hotel_df = hotel_df.drop(columns=['Unnamed: 0'])
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,Lüderitz,0,NaN,1619245938,56,-26.6481,15.1594,71.76,5.66,
125,Vila Velha,0,BR,1619245907,88,-20.3297,-40.2925,71.60,4.61,
177,Chiredzi,0,ZW,1619245958,52,-21.0500,31.6667,72.66,1.83,
290,Doaba,0,PK,1619245999,28,33.4244,70.7364,75.90,6.33,
422,Atar,0,MR,1619246050,20,20.5169,-13.0499,73.54,9.57,
449,Gwanda,0,ZW,1619246060,36,-20.9333,29.0000,71.06,5.41,
514,Kalat,0,PK,1619246085,12,29.0225,66.5916,73.47,8.59,
542,Saint-Louis,0,SN,1619246096,64,16.3333,-15.0000,71.60,2.30,


In [24]:
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row['City']
    params["location"] = f"{lat},{lng}"
    response = requests.get(url, params=params).json()
    results = response['results']
    
    try:
        print(f"The closest hotel to {city} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print('Error, not found...')

The closest hotel to Lüderitz is LÜDERITZ NEST HOTEL.
The closest hotel to Vila Velha is Hotel Senac Ilha do Boi.
The closest hotel to Chiredzi is The Nesbitt Arms.
Error, not found...
The closest hotel to Atar is Bab Sahara.
The closest hotel to Gwanda is Gwanda Hotel..
The closest hotel to Kalat is Balochistan Hotel.
Error, not found...


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))